In [1]:
import sys
import os
import json
from shapely.geometry import shape
import numpy as np
from dask.distributed import Client, LocalCluster
from pystac_client import Client as StacClient
import stackstac
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
## 1 Start a local Dask cluster
cluster = LocalCluster(n_workers=5, memory_limit="15GB")
client = Client(
    cluster,
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 15,Total memory: 69.85 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34901,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 15
Started: Just now,Total memory: 69.85 GiB
Comm: tcp://127.0.0.1:44629,Total threads: 3
Dashboard: http://127.0.0.1:36217/status,Memory: 13.97 GiB
Nanny: tcp://127.0.0.1:46175,


2025-11-28 22:49:21,543 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:42605' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('mean_chunk-b605bbf1ecd410bec02a2b43f2dfd4da', 95, 12, 4), ('any-aggregate-ebdc1051fdd67417aeb391300360a3ed', 94, 5, 1), ('any-aggregate-ebdc1051fdd67417aeb391300360a3ed', 95, 12, 6), ('getitem-d626cd826a1c2675360efb8b15fc9181', 95, 5, 4), ('getitem-ac1d65ec5a579b904fb757b45bf1d648', 93, 3, 6), ('getitem-ccac84d9deb8bc51c58e8b6a4ece97bb', 95, 5, 6), ('getitem-d626cd826a1c2675360efb8b15fc9181', 95, 11, 1), ('getitem-ccac84d9deb8bc51c58e8b6a4ece97bb', 95, 11, 3), ('getitem-d626cd826a1c2675360efb8b15fc9181', 95, 8, 2), ('getitem-d626cd826a1c2675360efb8b15fc9181', 95, 9, 1), ('any-aggregate-ebdc1051fdd67417aeb391300360a3ed', 94, 5, 3), ('mean_combine-partial-a9ff045077162f1baa7914a97eda14b1', 95, 1, 0), ('any-aggregate-ebdc1051fdd67417aeb391300360a3ed', 95, 9, 0), ('getitem-d626cd826a1c2675360efb8b15fc

In [3]:
os.getcwd()

'/home/jupyteruser/scr/notebooks'

In [3]:
# Add the folder containing WQI_utils.py
sys.path.append(os.path.abspath("../utils"))

## 2 Import the main function from wqi.py
from wqi_utils import qwi

In [5]:
bbox = [-82.8, 27.5, -82.2, 28.2]

wqi = qwi(
    bbox=bbox,
    start_date="2018-01-01",
    end_date="2022-12-31",
)
plot_qwi(wqi)


Number of Sentinel-2 scenes processed: 500


/opt/conda/envs/geog313-final-project/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


rededge5 not available; skipping NDCI


/opt/conda/envs/geog313-final-project/lib/python3.11/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 33.71 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2025-11-28 22:49:21,536 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:44629 -> tcp://127.0.0.1:42605
Traceback (most recent call last):
  File "/opt/conda/envs/geog313-final-project/lib/python3.11/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
                 ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/geog313-final-project/lib/python3.11/site-packages/tornado/iostream.py", line 1113, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
           ^^^^^^^^^^^^^^

KilledWorker: Attempted to run task ('fetch_raster_window-getitem-ccac84d9deb8bc51c58e8b6a4ece97bb', 94, 8, 4) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:41249. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

<Figure size 1200x600 with 0 Axes>

2025-11-28 22:53:09,950 - distributed.nanny - WARNING - Restarting worker
2025-11-28 22:53:09,990 - distributed.nanny - WARNING - Restarting worker


In [7]:
wqi_time_series(
    bbox=bbox,
    start_date=START_DATE,
    end_date=END_DATE,
    filter_clouds=True,          
)

AssertionError: out_bounds=None

In [ ]:
main(
    bbox=bbox,
    start_date=START_DATE,
    end_date=END_DATE,
    filter_clouds=False,
)

In [11]:
from pystac_client import Client

client = Client.open("https://earth-search.aws.element84.com/v1")
items = client.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime=f"{START_DATE}/{END_DATE}",
    max_items=500
).item_collection()

print("Number of items found:", len(items))


Number of items found: 500


In [5]:
wqi_dict = wqi(
    bbox=bbox,
    start_date=START_DATE,
    end_date=END_DATE,
    filter_clouds=True,          
)

AssertionError: out_bounds=None